# Produces dataset of project times from appended pipeline datasets ("all_quarters_merged.csv")

In [ ]:
import pandas as pd
import logging
import dateutil
from dateutil import parser
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x) 

In [ ]:
logging.basicConfig(level=logging.INFO)

# Code Broken Down

In [ ]:
df = pd.read_csv("cleaned/all_quarters_merged.csv")

In [ ]:
# keep only variables of interest
list = ['address', 'apn', 'best_date', 'best_stat','firstfiled', 'report_quarter', 'report_year', 'units', 'unitsnet', 'dbi_permit', 'x', 'y']
df = df[list]

In [ ]:
#consolidate status categories. Start with 3 for now
def status_function(value):
    if value['best_stat']=="CONSTRUCTION":
        field = 'Under Construction'
    elif (value['best_stat']=='BP APPROVED') | (value['best_stat']=='BP ISSUED') | (value['best_stat']=='BP REINSTATED'):
        field = 'Building Permit Approved'
    else:
        field = 'Proposed'
    return field
        
df['status']=df.apply(status_function, axis=1)

In [ ]:
gb = df.groupby(['apn', 'address'])

In [ ]:
gb.groups

In [ ]:
group_df = gb.get_group(('6019006','340 NAPLES ST'))

In [ ]:
group_df=group_df.sort_values(['best_date', 'report_year', 'report_quarter'], ascending=[True, True, True])
group_df.reset_index(drop=True, inplace=True)

In [ ]:
group_df

In [ ]:
last_row = group_df.tail(1).copy()
        
# identify building permit ID. Then fill in rest of quarters with this permit ID.
building_permit=np.nan
for index, row in group_df.iterrows():
    if pd.isnull(row['dbi_permit']) and not pd.isnull(building_permit):
        building_permit=building_permit
    else:
        building_permit = row['dbi_permit']

#Identify completion quarter for those projects that have reached completion
for index, row in group_df.iterrows():
    if row['best_stat'] !='CONSTRUCTION':
        comp_quarter= np.nan
        comp_year=np.nan
    elif row['best_stat'] == 'CONSTRUCTION':
        comp_quarter = row['report_quarter']
        comp_year = row['report_year']

if pd.isnull(comp_quarter):
    pass
if comp_quarter == 1 and comp_year == 2016:
    comp_quarter = np.nan
elif comp_quarter == 4:
    comp_quarter = 1
    comp_year = 1+comp_year
else:
    comp_quarter = 1+comp_quarter

if comp_quarter ==1:
    comp_daymth= '01/01'
elif comp_quarter ==2:
    comp_daymth= '04/01'
elif comp_quarter==3:
    comp_daymth= '07/01'
elif comp_quarter == 4:
    comp_daymth= '10/01'
elif pd.isnull(comp_quarter):
    comp_daymth=np.nan

if pd.isnull(comp_daymth):
    comp_date = np.nan
else:
    comp_date = comp_daymth + "/" + str(comp_year)

#Identify earliest "firstfiled" date
firstfiled=''
for index, row in group_df.iterrows():
    if pd.isnull(row['firstfiled']):
        continue
    else:
        if len(firstfiled) ==0:
            firstfiled=row['firstfiled']
        else:
            if dateutil.parser.parse(row['firstfiled']) < dateutil.parser.parse(firstfiled):
                firstfiled=row['firstfiled']
            else: 
                firstfiled=firstfiled
        
#Identify earliest best date
earliest_BD = ''
for index, row in group_df.iterrows():
    if pd.isnull(row['best_date']):
        continue
    else:
        if len(earliest_BD) == 0:
            earliest_BD = row['best_date']
        else:
            continue

# Finalize first date variable (minimum of earliest best_date and firstfiled)
if firstfiled =='' and earliest_BD !='':
    first_date = earliest_BD
elif earliest_BD=='' and firstfiled !='':
    first_date = firstfiled
elif firstfiled !='' and earliest_BD !='':
    first_date = min(firstfiled, earliest_BD)

# initiate variables. Groups without these dates are blank for these variables.
BP_date = ''
con_date = ''

#Identify first date for all status categories
m=0
for index, row in group_df.iterrows():
    if m == 0:
        status_previous = 'blah'
    if row['status']=='Building Permit Approved':
        if index == 0:
            BP_date = row['best_date']
        elif index !=0:
            if status_previous =='Building Permit Approved':
                BP_date=BP_date
            else:
                BP_date = row['best_date']
    elif row['status']=='Under Construction':
        if index == 0:
            con_date = row['best_date']
        elif index !=0:
            if status_previous =='Under Construction':
                con_date=con_date
            else:
                con_date = row['best_date']
    status_previous = row['status']
    m=m+1

#Identify latest unit counts
units = np.nan
for index, row in group_df.iterrows():
    if pd.isnull(row['units']):
        continue
    else:
        units = row['units']
                
#Identify latest net unit counts
unitsnet= np.nan
for index, row in group_df.iterrows():
    if pd.isnull(row['unitsnet']):
        continue
    else:
        unitsnet = row['unitsnet']

#a few projects have construction best date after the completion date. In these cases, match the two.
if pd.notnull(comp_date):
    if dateutil.parser.parse(comp_date) < dateutil.parser.parse(con_date):
        comp_date = con_date
        
last_row['firstfiled']=firstfiled
last_row['dbi_permit']= building_permit
last_row['comp_date']=comp_date
last_row['BP_date'] = BP_date
last_row['con_date'] = con_date
last_row['first_date']=first_date
last_row['latest_project_record_date'] = last_row.best_date
last_row['first_project_record_date'] = group_df.iloc[0].best_date
last_row['latest_project_status'] = last_row.best_stat
last_row['units']=units
last_row['unitsnet']=unitsnet

## Store a parseable list of all the project states and the dates those states were reported
last_row['project_dates'] = str(tuple(group_df.best_date))
last_row['project_statuses'] = str(tuple(group_df.best_stat))

## Store the project duration in days
if not (pd.isnull(comp_date) or pd.isnull(first_date)):
    last_row['project_duration_days'] = (dateutil.parser.parse(comp_date) - dateutil.parser.parse(first_date)).days

In [ ]:
print (type(comp_date))

In [ ]:
print (type(con_date))

In [ ]:
comp_date

In [ ]:
con_date

In [ ]:
last_row

# Actual Code

In [ ]:
def convert_to_one_record_per_project(df):
    """
    Group the dataset by (apn, address) and then emit one row per (apn, address) pair.
    best_date and best_stat will be converted into arrays.
    All other attributes will be taken from the record with the most recent `best_date` attribute.
    """
    gb = df.groupby(['apn', 'address'])
    for k in gb.groups:
        group_df = gb.get_group(k)
        group_df = group_df.sort_values(['best_date', 'report_year', 'report_quarter'], ascending=[True, True, True])
        last_row = group_df.tail(1).copy()
        
        # identify building permit ID. Then fill in rest of quarters with this permit ID.
        building_permit=np.nan
        for index, row in group_df.iterrows():
            if pd.isnull(row['dbi_permit']) and not pd.isnull(building_permit):
                building_permit=building_permit
            else:
                building_permit = row['dbi_permit']
                
        #Identify completion quarter for those projects that have reached completion
        for index, row in group_df.iterrows():
            if row['best_stat'] !='CONSTRUCTION':
                comp_quarter= np.nan
                comp_year=np.nan
            elif row['best_stat'] == 'CONSTRUCTION':
                comp_quarter = row['report_quarter']
                comp_year = row['report_year']

        if pd.isnull(comp_quarter):
            pass
        if comp_quarter == 1 and comp_year == 2016:
            comp_quarter = np.nan
        elif comp_quarter == 4:
            comp_quarter = 1
            comp_year = 1+comp_year
        else:
            comp_quarter = 1+comp_quarter
            
        if comp_quarter ==1:
            comp_daymth= '01/01'
        elif comp_quarter ==2:
            comp_daymth= '04/01'
        elif comp_quarter==3:
            comp_daymth= '07/01'
        elif comp_quarter == 4:
            comp_daymth= '10/01'
        elif pd.isnull(comp_quarter):
            comp_daymth=np.nan

        if pd.isnull(comp_daymth):
            comp_date = np.nan
        else:
            comp_date = comp_daymth + "/" + str(comp_year)
        
        #Identify earliest "firstfiled" date
        firstfiled=''
        for index, row in group_df.iterrows():
            if pd.isnull(row['firstfiled']):
                continue
            else:
                if len(firstfiled) ==0:
                    firstfiled=row['firstfiled']
                else:
                    if dateutil.parser.parse(row['firstfiled']) < dateutil.parser.parse(firstfiled):
                        firstfiled=row['firstfiled']
                    else: 
                        firstfiled=firstfiled
        
        #Identify earliest best date
        earliest_BD = ''
        for index, row in group_df.iterrows():
            if pd.isnull(row['best_date']):
                continue
            else:
                if len(earliest_BD) == 0:
                    earliest_BD = row['best_date']
                else:
                    continue
        
        # Finalize first date variable (minimum of earliest best_date and firstfiled)
        if firstfiled =='' and earliest_BD !='':
            first_date = earliest_BD
        elif earliest_BD=='' and firstfiled !='':
            first_date = firstfiled
        elif firstfiled !='' and earliest_BD !='':
            first_date = min(firstfiled, earliest_BD)
            
        # initiate variables. Groups without these dates are blank for these variables.
        BP_date = ''
        con_date = ''

        #Identify first date for all status categories
        m=0
        for index, row in group_df.iterrows():
            if m == 0:
                status_previous = 'blah'
            if row['status']=='Building Permit Approved':
                if index == 0:
                    BP_date = row['best_date']
                elif index !=0:
                    if status_previous =='Building Permit Approved':
                        BP_date=BP_date
                    else:
                        BP_date = row['best_date']
            elif row['status']=='Under Construction':
                if index == 0:
                    con_date = row['best_date']
                elif index !=0:
                    if status_previous =='Under Construction':
                        con_date=con_date
                    else:
                        con_date = row['best_date']
            status_previous = row['status']
            m=m+1
            
        #Identify latest unit counts
        units = np.nan
        for index, row in group_df.iterrows():
            if pd.isnull(row['units']):
                continue
            else:
                units = row['units']
                
        #Identify latest net unit counts
        unitsnet= np.nan
        for index, row in group_df.iterrows():
            if pd.isnull(row['unitsnet']):
                continue
            else:
                unitsnet = row['unitsnet']
                
        #a few projects have construction best date after the completion date. In these cases, match the two.
        if pd.notnull(comp_date):
            if dateutil.parser.parse(comp_date) < dateutil.parser.parse(con_date):
                comp_date = con_date
        
        last_row['firstfiled']=firstfiled
        last_row['dbi_permit']= building_permit
        last_row['comp_date']=comp_date
        last_row['BP_date'] = BP_date
        last_row['con_date'] = con_date
        last_row['first_date']=first_date
        last_row['latest_project_record_date'] = last_row.best_date
        last_row['first_project_record_date'] = group_df.iloc[0].best_date
        last_row['latest_project_status'] = last_row.best_stat
        last_row['units']=units
        last_row['unitsnet']=unitsnet

        ## Store a parseable list of all the project states and the dates those states were reported
        last_row['project_dates'] = str(tuple(group_df.best_date))
        last_row['project_statuses'] = str(tuple(group_df.best_stat))

        ## Store the project duration in days
        if not (pd.isnull(comp_date) or pd.isnull(first_date)):
            last_row['project_duration_days'] = (dateutil.parser.parse(comp_date) - dateutil.parser.parse(first_date)).days

        yield last_row


def main():
    df = pd.read_csv("cleaned/all_quarters_merged.csv")
    #consolidate status categories. Start with 3 for now
    def status_function(value):
        if value['best_stat']=="CONSTRUCTION":
            field = 'Under Construction'
        elif (value['best_stat']=='BP APPROVED') | (value['best_stat']=='BP ISSUED') | (value['best_stat']=='BP REINSTATED'):
            field = 'Building Permit Approved'
        else:
            field = 'Proposed'
        return field

    df['status']=df.apply(status_function, axis=1)
    
    #keep only variables we want
    list = ['address', 'apn', 'best_date', 'best_stat','status', 'firstfiled', 'report_quarter', 'report_year','units','unitsnet', 'dbi_permit', 'x', 'y']
    df = df[list]
    new_df = pd.concat(convert_to_one_record_per_project(df))
    logging.info("Writing output ({} rows, {} cols) to data/cleaned/all_quarters__one_record_per_project.csv".format(*new_df.shape))
    new_df.to_csv("cleaned/all_quarters__one_record_per_project.csv")
    return new_df

In [ ]:
new_df = main()

In [ ]:
#First, filter out those projects that are exclusively non-residential (defined as those without units)
new_df = new_df[new_df['units'] > 0]

In [ ]:
#Next, keep only those that reached completion at some point over the time period
new_df=new_df[pd.notnull(new_df['comp_date'])]

# Explore Duplicate Data

In [ ]:
new_df['address_apn']=new_df['address']+new_df['apn']

In [ ]:
#Finally, filter out records with duplicate BP Ids
new_df[new_df.duplicated('dbi_permit', keep=False)]['address_apn'].unique()


In [ ]:
new_df[new_df['address_apn']=='1634 - 1690 PINE ST0647007']['dbi_permit']

In [ ]:
# Same project, different address-apn combination—verified by looking at each address in PIM
new_df[new_df['dbi_permit']==201312184508.00]

In [ ]:
#explore duplicated
new_df[new_df['address']=='2155 WEBSTER ST']

In [ ]:
new_df[new_df['dbi_permit']=='200705010136']

In [ ]:
new_df[new_df['project_duration_days']<0]

# Manual Data Cleaning
Based off of identified duplicates in the previous section

In [ ]:
#decided that both this and its duplicate BP at 1169 Market Street are two phases of the same project. Same start date, different end dates
new_df.loc[(new_df['address']=='1190 MISSION ST') & (new_df['apn']=='3702052'), 'first_date'] = '2003-07-15'

In [ ]:
#this has a duplicate, same address, different APN
new_df=new_df[(new_df['address']!='3575 GEARY BL') | (new_df['apn']=='1084010')]

In [ ]:
#same project, happening at different periods
new_df.loc[(new_df['address']=='1634-1690 PINE ST') & (new_df['apn']=='0647007'), 'comp_date']='07/01/2016'
new_df=new_df[(new_df['address']!='1634 - 1690 PINE ST') | (new_df['apn']=='0647007')]

In [ ]:
new_df.loc[(new_df['address']=='1634-1690 PINE ST') & (new_df['apn']=='0647007')]

In [ ]:
#Be sure to redo project duration after all manual cleaning

In [ ]:
#do median time based on neighborhood

In [ ]:
#breakdown time to completion by stage for those projects that this can be done for